# Hybrid & Vector Search - Ungraph

Este notebook demuestra cómo usar búsqueda vectorial y búsqueda híbrida en Ungraph.

## Objetivos

1. **Búsqueda vectorial pura** - Búsqueda por similitud semántica usando embeddings
2. **Búsqueda híbrida** - Combinación de texto y vectorial para mejores resultados
3. **Ajuste de pesos** - Cómo ajustar la importancia de texto vs vectorial
4. **Comparación de métodos** - Cuándo usar cada tipo de búsqueda
5. **Ejemplos prácticos** - Casos de uso reales

## Tipos de Búsqueda

- **Text Search**: Búsqueda por palabras clave (full-text)
- **Vector Search**: Búsqueda por similitud semántica (embeddings)
- **Hybrid Search**: Combinación de ambas para mejores resultados

**Referencias:**
- [Guía de Búsqueda](../../docs/guides/search.md)
- [API Pública](../../docs/api/public-api.md)


In [ ]:
def add_src_to_path(path_folder: str):
    ''' 
    Helper function for adding the "path_folder" directory to the path.
    '''
    import sys
    from pathlib import Path

    base_path = Path().resolve()
    for parent in [base_path] + list(base_path.parents):
        candidate = parent / path_folder
        if candidate.exists():
            parent_dir = candidate.parent
            if str(parent_dir) not in sys.path:
                sys.path.insert(0, str(parent_dir))
            if str(candidate) not in sys.path:
                sys.path.append(str(candidate))
            return

# Agregar carpetas necesarias al path
add_src_to_path(path_folder="src")
add_src_to_path(path_folder="src/utils")
add_src_to_path(path_folder="src/data")


In [ ]:
# Importar librerías necesarias
import sys
from pathlib import Path
from typing import List

# Importar ungraph
try:
    import ungraph
    print("✅ Ungraph importado como paquete instalado")
except ImportError:
    import src
    ungraph = src
    print("✅ Ungraph importado desde src/ (modo desarrollo)")

# Importar servicios específicos
from infrastructure.services.neo4j_search_service import Neo4jSearchService
from infrastructure.services.huggingface_embedding_service import HuggingFaceEmbeddingService

print(f"📦 Ungraph version: {ungraph.__version__}")


## Parte 1: Configuración y Datos de Prueba

Primero necesitamos tener datos en el grafo. Si no tienes datos, puedes usar el notebook de ingesta básica primero.


In [ ]:
# Verificar que tenemos datos en el grafo
from src.utils.graph_operations import graph_session

driver = graph_session()
try:
    with driver.session() as session:
        result = session.run("MATCH (c:Chunk) RETURN count(c) as count")
        count = result.single()["count"]
        print(f"📊 Chunks en el grafo: {count}")
        
        if count == 0:
            print("⚠️  No hay chunks en el grafo. Por favor ingiere documentos primero.")
            print("   Puedes usar el notebook '0. Basic Ingestion.ipynb' para ingerir datos.")
        else:
            print("✅ Datos disponibles para búsqueda")
finally:
    driver.close()


## Parte 2: Búsqueda Vectorial Pura

La búsqueda vectorial usa embeddings para encontrar contenido semánticamente similar, sin depender de palabras clave exactas.


In [ ]:
# 1. Crear servicio de embeddings
embedding_service = HuggingFaceEmbeddingService()

# 2. Generar embedding de la query
query_text = "inteligencia artificial y machine learning"
print(f"🔍 Query: '{query_text}'")
print("📊 Generando embedding...")

query_embedding = embedding_service.generate_embedding(query_text)
print(f"✅ Embedding generado: {len(query_embedding.vector)} dimensiones")
print(f"   Modelo: {query_embedding.encoder_info}")


In [ ]:
# 3. Realizar búsqueda vectorial
search_service = Neo4jSearchService()

print("🔍 Realizando búsqueda vectorial...")
vector_results = search_service.vector_search(
    query_embedding=query_embedding,
    limit=5
)

print(f"\n✅ Encontrados {len(vector_results)} resultados:\n")
for i, result in enumerate(vector_results, 1):
    print(f"Resultado {i}:")
    print(f"  Score: {result.score:.4f}")
    print(f"  Chunk ID: {result.chunk_id}")
    print(f"  Contenido: {result.content[:200]}...")
    if result.previous_chunk_content:
        print(f"  Contexto anterior: {result.previous_chunk_content[:100]}...")
    if result.next_chunk_content:
        print(f"  Contexto siguiente: {result.next_chunk_content[:100]}...")
    print()

search_service.close()


## Parte 3: Búsqueda Híbrida

La búsqueda híbrida combina búsqueda por texto (full-text) y búsqueda vectorial para obtener mejores resultados.


In [ ]:
# Búsqueda híbrida usando la función de alto nivel
query_text = "deep learning y redes neuronales"

print(f"🔍 Query: '{query_text}'")
print("🔍 Realizando búsqueda híbrida (pesos por defecto: 30% texto, 70% vectorial)...\n")

hybrid_results = ungraph.hybrid_search(
    query_text=query_text,
    limit=5,
    weights=(0.3, 0.7)  # 30% texto, 70% vectorial
)

print(f"✅ Encontrados {len(hybrid_results)} resultados:\n")
for i, result in enumerate(hybrid_results, 1):
    print(f"Resultado {i}:")
    print(f"  Score combinado: {result.score:.4f}")
    print(f"  Chunk ID: {result.chunk_id}")
    print(f"  Contenido: {result.content[:200]}...")
    print()


## Parte 4: Ajuste de Pesos en Búsqueda Híbrida

Los pesos determinan qué tan importante es cada tipo de búsqueda. Puedes ajustarlos según tus necesidades.


In [ ]:
# Comparar diferentes combinaciones de pesos
query_text = "computación cuántica"

weight_configs = [
    (0.7, 0.3, "Más peso a texto (palabras clave exactas)"),
    (0.5, 0.5, "Balanceado"),
    (0.3, 0.7, "Más peso a vectorial (conceptos semánticos) - DEFAULT"),
    (0.2, 0.8, "Muy enfocado en semántica"),
]

print(f"🔍 Query: '{query_text}'\n")
print("=" * 80)

for text_weight, vector_weight, description in weight_configs:
    print(f"\n📊 Configuración: {description}")
    print(f"   Pesos: {text_weight*100:.0f}% texto, {vector_weight*100:.0f}% vectorial")
    
    results = ungraph.hybrid_search(
        query_text=query_text,
        limit=3,
        weights=(text_weight, vector_weight)
    )
    
    print(f"   Top 3 resultados:")
    for i, result in enumerate(results, 1):
        print(f"     {i}. Score: {result.score:.4f} | {result.content[:80]}...")


## Parte 5: Comparación de Métodos

Comparemos los tres tipos de búsqueda con la misma query.


In [ ]:
# Comparar los tres métodos
query_text = "machine learning applications"

print(f"🔍 Query: '{query_text}'\n")
print("=" * 80)

# 1. Búsqueda por texto
print("\n1️⃣ BÚSQUEDA POR TEXTO (Full-text)")
print("-" * 80)
text_results = ungraph.search(query_text, limit=3)
for i, result in enumerate(text_results, 1):
    print(f"  {i}. Score: {result.score:.4f} | {result.content[:80]}...")

# 2. Búsqueda vectorial
print("\n2️⃣ BÚSQUEDA VECTORIAL (Semántica)")
print("-" * 80)
embedding_service = HuggingFaceEmbeddingService()
query_embedding = embedding_service.generate_embedding(query_text)
search_service = Neo4jSearchService()
vector_results = search_service.vector_search(query_embedding, limit=3)
for i, result in enumerate(vector_results, 1):
    print(f"  {i}. Score: {result.score:.4f} | {result.content[:80]}...")
search_service.close()

# 3. Búsqueda híbrida
print("\n3️⃣ BÚSQUEDA HÍBRIDA (Texto + Vectorial)")
print("-" * 80)
hybrid_results = ungraph.hybrid_search(query_text, limit=3, weights=(0.3, 0.7))
for i, result in enumerate(hybrid_results, 1):
    print(f"  {i}. Score: {result.score:.4f} | {result.content[:80]}...")


## Parte 6: Cuándo Usar Cada Tipo de Búsqueda

### Búsqueda por Texto
- ✅ **Cuándo usar**: Búsquedas por palabras clave exactas, términos técnicos específicos
- ✅ **Ventajas**: Muy rápida, buena para términos exactos
- ❌ **Limitaciones**: No captura sinónimos o conceptos relacionados

### Búsqueda Vectorial
- ✅ **Cuándo usar**: Conceptos abstractos, búsquedas semánticas, sinónimos
- ✅ **Ventajas**: Entiende significado, encuentra contenido relacionado
- ❌ **Limitaciones**: Puede ser más lenta, requiere embeddings

### Búsqueda Híbrida
- ✅ **Cuándo usar**: La mayoría de casos de uso, mejor precisión general
- ✅ **Ventajas**: Combina lo mejor de ambos mundos
- ❌ **Limitaciones**: Más compleja, requiere ambos índices

## Mejores Prácticas

1. **Empezar con híbrida**: Generalmente da mejores resultados
2. **Ajustar pesos según necesidad**: 
   - Más texto (0.7, 0.3) para palabras clave exactas
   - Más vectorial (0.2, 0.8) para conceptos abstractos
3. **Usar límites razonables**: `limit=5-10` suele ser suficiente
4. **Reconstruir contexto**: Usa chunks adyacentes para mejor comprensión

## Referencias

- [Guía de Búsqueda](../../docs/guides/search.md)
- [API Pública](../../docs/api/public-api.md)
- [Patrones de Búsqueda GraphRAG](../../docs/api/search-patterns.md)
